# Dynamic WSCC 9-bus System with Switch Event

**Authors**:
 - Markus Mirz <mmirz@eonerc.rwth-aachen.de>
 - Steffen Vogel <stvogel@eoner.rwth-aachen.de>

This Jupyter Notebook shows a simple dynamic phasor simulation of the WSCC-9bus benchmark model.
The grid data is loaded from CIM-XML files, while simulation results are stored in CSV files and plotted via Matplotlib

In [ ]:
%gui asyncio
#%matplotlib widget

import dpsim
from dpsim.Event import Event
import glob
import asyncio
import matplotlib.pyplot as plt

### Loading Grid Topology from CIM-XML Model

In [ ]:
name = 'WSCC-9bus_dyn_switch'
files = glob.glob('../../CIM/grid-data/WSCC-09/WSCC-09_RX_Dyn/*.xml') + \
        glob.glob('Examples/CIM/grid-data/WSCC-09/WSCC-09_RX_Dyn/*.xml')
print(files)
system = dpsim.load_cim(name, files, frequency=60)

### Rending Network Topology via Graphiz/SVG

We use Graphviz to render the network model into a layouted SVG figure

In [ ]:
system

### Extending Network with Switch and Load

Here we add a new switch to Bus 9 that is triggered at 0.05s. This component could be added permantely to the CIM model as well using tools like Pintura.

In [ ]:
## Switch
sw = dpsim.dp.ph1.Switch("Switch")
sw.R_open = 1e9
sw.R_closed = 0.1
sw.is_closed = False

## Load
load = dpsim.dp.ph1.PQLoadCS("Switched Load")
load.V_nom = 230950
load.P = 30000000
load.Q = 0

## Topology
bus9 = system.nodes["BUS6"]
gnd = dpsim.dp.Node.GND()

sw.connect([ bus9, gnd ])

system.add_component(sw)

In [ ]:
system

### Running Simulation

The actual simulation is done by the C++ DPsim solver. Python is just used for configuration, scripting and analysis

In [ ]:
sim = dpsim.Simulation(name, system, timestep=0.0001, duration=2, init_steady_state=True, pbar=True)

#system.components['GEN3'].inertia *= 2
sw.is_closed = False
sim.add_event(0.2, sw, 'is_closed', True)

logger = dpsim.Logger(name)
sim.add_logger(logger)
for i in range(1,4):
	logger.log_attribute(system.components['GEN%d' % i], 'w_r')

for node in system.nodes:
	logger.log_attribute(system.nodes[node], 'v')

sim.start()

## Analysis

### Read log files and list all column names

In [ ]:
import matplotlib.pyplot as plt
import villas.dataprocessing.plottools as pt
import villas.dataprocessing.readtools as rt
from villas.dataprocessing.timeseries import TimeSeries as ts
import numpy as np

res = rt.read_timeseries_dpsim('logs/WSCC-9bus_dyn_switch.csv')

### Phasors at first time step

In [ ]:
phasors = ts.phasors(res)
for node, phasor in phasors.items():
    if 'v' in node:
        print(node + ': ' + str(phasor['abs'].values[0]) + '<' + str(phasor['phase'].values[0]))

### Phasors at last time step

In [ ]:
for node, phasor in phasors.items():
    if 'v' in node:
        print(node + ': ' + str(phasor['abs'].values[-1]) + '<' + str(phasor['phase'].values[-1]))

### Phasors at last time step in per unit

In [ ]:
nominal_voltages = {
    'BUS1.v': 16.5e3,
    'BUS2.v': 18e3,
    'BUS3.v': 13.8e3, 
    'BUS4.v': 230e3,
    'BUS5.v': 230e3,
    'BUS6.v': 230e3, 
    'BUS7.v': 230e3,
    'BUS8.v': 230e3,
    'BUS9.v': 230e3
}

plt.figure(1)
for node, nom_voltage in nominal_voltages.items():
    mag = phasors[node]['abs'].values[0] / nom_voltage
    pha = phasors[node]['phase'].values[0]
    print(node + ': ' + str(mag) + '<' + str(pha))
    plt.polar([0, pha / 180 * np.pi], [0, mag], marker='o', label=node)
plt.show()

### Plot node phases

In [ ]:
for i in range(1,9):
    pt.plot_timeseries(20, phasors['BUS%d.v' % i]['phase'])

### Plot node voltages

In [ ]:
for i in range(4,9):
    pt.plot_timeseries(10, phasors['BUS%d.v' % i]['abs'])
for i in range(1,4):
    pt.plot_timeseries(11, phasors['BUS%d.v' % i]['abs'])
plt.xlim(0.0, 0.06)

In [ ]:
res['GEN1.w_r'].label = 'GEN1 wr'
res['GEN2.w_r'].label = 'GEN2 wr'
res['GEN3.w_r'].label = 'GEN3 wr'
pt.plot_timeseries(1, res['GEN1.w_r'])
pt.plot_timeseries(1, res['GEN2.w_r'])
pt.plot_timeseries(1, res['GEN3.w_r'])
plt.xlabel('time (s)')
plt.ylabel('mechanical speed (rad/s)')

## Validation

In [ ]:
# read Simulink log file
import os
import urllib.request

if not os.path.exists('reference-results'):
    os.mkdir('reference-results')

url = 'https://git.rwth-aachen.de/acs/public/simulation/dpsim-results/raw/master/WSCC-9bus/WSCC-9bus_dyn_switch.csv'
local_file = 'reference-results/WSCC-9bus_dyn_switch.csv'
urllib.request.urlretrieve(url, local_file) 

ts_sl = rt.read_timeseries_simulink(local_file)

In [ ]:
import numpy as np
assert np.all(ts_sl['GEN1.w_r'].values - res['GEN1.w_r'].values < 0.001)